<a href="https://colab.research.google.com/github/jscienciadados/banco-sql/blob/main/python_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Banco de Dados

* Componentes
  * Base de dados
  * Sistema Gerenciador de Bando de Dados (SGDB)
  * Linguagem de Manipulação
    *  Structered Query Language (SQL)
  * Programas adicionais

* Arquitetura
  * Cliente-Servidor

## Modelo Relacional
  
| nome | tipo | level | capturado em|
|---- |----|-----|----|
|Pikachu| Elétrico| 5| 08 jun 2021|
|Bulbassauro| Grama| 10| 10 fev 1990|
|Squirtle| Água| 12| 23 out 2015|

Tabela ou Relação

Linhas, Intâncias, Entradas

Colunas, atributos, caracteríísticas

## Implementação em SQL

* dialetos: MySQL, PostreSQL, MariaDB, Oracle etc

* MySQL online: https://paiza.io/en/projects/new?language=mysql
* PostgreSQL as a Service: https://www.elephantsql.com/
* Sqlite e PostgreSQL: https://sqliteonline.com/

#### Criando um banco de dados dentro do SGDB
* MySQL
      CREATE {DATABASE} [IF NOT EXISTS] db_name
*PostgreSQL
      CREATE DATABASE name
*SQLite
      # Não tem (nesse formato)

### Linguagem de Descrição de Dados (DDL)


#### CREATE (Criar)
      CREATE [TEMPORARY] TABLE [IF NOT EXISTS] tbl_name
      [column_name column_type column_constraint, ...]
      [table_constraints, ...]

[MySQL types](https://dev.mysql.com/doc/refman/8.0/en/data-types.html) 
\- [PostrgreSQL types](https://www.postgresql.org/docs/9.5/datatype.html)
\- [SQLite types](https://www.sqlite.org/datatype3.html)

* Most used Constraints:

#### ALTER (alterar)
      ALTER TABLE tbl_name [alter_option, ...]

#### DROP (descartar)
      DROP TABLE [ IF EXISTS ] tbl_name [, ...]


### Linguagem de Manipulação de Dados (DML)


#### INSERT (inserir)

      INSERT  INTO tbl_name[(col_name [, col_name] ...)]
      VALUES (value_list) [, (value_list)] ...```


#### UPDATE (atualizar)
      UPDATE table_reference
      SET assignment_list
      [WHERE where_condition]

#### DELETE (deletar) 
      DELETE FROM tbl_name [WHERE where_condition]

### Linguagem de Consulta de Dados (DQL)


#### SELECT (SELECIONAR)

      SELECT [ALL | ISTINCT | DISTINCTROW ]
      select_expr [, select_expr] ...
      [FROM table_references
      [WHERE where_condition]
      [GROUP BY {col_name | expr | position}, ...]
      [HAVING where_condition]
      [ORDER BY {col_name | expr | position} [ASC | DESC], ...]
      [LIMIT {[offset,] row_count | row_count OFFSET offset}]

#### JOINS

### Linguagem de Transação de Dados (DTL)
### Linguagem de Controle de Dados (DCL)

# Integração com Python


In [ ]:
import sqlite3

## SQLAlchemy

É um *framework* desenvolvido para permitir programar aplicações que usem Banco de dados, de maneira inpedepende e com diferentes tipos de abstração:

* Raw SQL (SQL Bruto)
* SQL Expression Language (Linguagem de Expressão SQL)
* Object-Relational Mapping - ORM (Mapeamento Objeto-Relacional)

### Raw SQL

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.sql import text

In [ ]:
# Cria a engine para o banco de dados
eng = create_engine("sqlite:///cars.db")

In [ ]:
# abre uma conexão com o banco e executa código SQL
with eng.connect() as con:
  con.execute(text('DROP TABLE IF EXISTS Cars'))
  con.execute(text('''CREATE TABLE Cars(Id INTEGER PRIMARY KEY, 
                   Name TEXT, Price INTEGER)'''))

In [ ]:
data = ({ "Id": 1, "Name": "Audi", "Price": 52642 },
        { "Id": 2, "Name": "Mercedes", "Price": 57127 },
        { "Id": 3, "Name": "Skoda", "Price": 9000 },
        { "Id": 4, "Name": "Volvo", "Price": 29000 },
        { "Id": 5, "Name": "Bentley", "Price": 350000 },
        { "Id": 6, "Name": "Citroen", "Price": 21000 },
        { "Id": 7, "Name": "Hummer", "Price": 41400 },
        { "Id": 8, "Name": "Volkswagen", "Price": 21600 }
)
# Inserindo dados
with eng.connect() as con:
  for line in data:
    con.execute(text("""INSERT INTO Cars(Id, Name, Price) VALUES(:Id, :Name, :Price)"""), **line)

In [ ]:
# Recebendo dados
with eng.connect() as con:
  rs = con.execute(text("SELECT * FROM Cars"))
  print(rs.fetchone())
  print(rs.fetchmany(3))
  print(rs.fetchall())

### SQL Expression Language

#### Definição de Schemas

In [ ]:
from sqlalchemy import Table, Column, Integer, String, MetaData

In [ ]:
# Cria um ojeto de metadata. Metadatas sãão coleçoes de tabelas e sua infos
meta = MetaData()

In [ ]:
#Cria um objeto do tipo tabela que abstrai uma tabela no Banco de dados
cars = Table('Cars', meta,
     Column('Id', Integer, primary_key=True),
     Column('Name', String),
     Column('Price', Integer)
)

In [ ]:
print "The Name column:"
print cars.columns.Name
print cars.c.Name

print "Columns: "
for col in cars.c:
    print col
    
print "Primary keys:"
for pk in cars.primary_key:
    print pk    

print "The Id column:"
print cars.c.Id.name
print cars.c.Id.type
print cars.c.Id.nullable
print cars.c.Id.primary_key

In [ ]:
meta.reflect(bind=eng)
for table in meta.tables:
    print(table)

In [ ]:
from sqlalchemy import nspect

In [ ]:
insp = inspect(eng)
print(insp.get_table_names())
print(insp.get_columns("Cars"))
print(insp.get_pk_constraint("Cars")) 

#### Expression Language

In [ ]:
from sqlalchemy.sql import select 

In [ ]:
meta = MetaData(eng)
cars2 = Table('Cars', meta, autoload=True)  


In [ ]:
# SELECT
with eng.connect() as con:
  stm = select([cars])
  rs = con.execute(stm) 
  print(rs.fetchall())

In [ ]:
# LIMIT
with eng.connect() as con:
  stm = select([cars.c.Name, cars.c.Price]).limit(3)
  rs = con.execute(stm) 
  print(rs.fetchall())

In [ ]:
from sqlalchemy.sql import and_, or_  

In [ ]:
# WHERE
with eng.connect() as con:
  stm = select([cars]).where(and_(cars.c.Price > 10000, cars.c.Price < 40000))
  rs = con.execute(stm) 
  print(rs.fetchall())

In [ ]:
#LIKE
with eng.connect() as con:
  stm = select([cars]).where(cars.c.Name.like('%en'))
  rs = con.execute(stm) 
  print(rs.fetchall())

In [ ]:
from sqlalchemy.sql import asc, desc

In [ ]:
# Order By
with eng.connect() as con:  
  s = select([cars]).order_by(asc(cars.c.Name))
  rs = con.execute(s) 
  for row in rs:
    print(row['Id'], row['Name'], row['Price'])

In [ ]:

# ler dos arquivos
with eng.connect() as con:
  authors = Table('Authors', meta, autoload=True)
  books = Table('Books', meta, autoload=True)
  stm = select([authors.join(books)])
  rs = con.execute(stm) 
  for row in rs:
    print row['Name'], row['Title']

### Object-Relational Mapping (ORM)

In [ ]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import sessionmaker

In [ ]:
# Classes Bases e Sessão
Base = declarative_base(bind=eng)
Session = sessionmaker(bind=eng)

In [ ]:
class Car(Base):
  __tablename__ = "Cars"

  id = Column(Integer, primary_key=True)
  name = Column(String)  
  price = Column(Integer)

In [ ]:
Base.metadata.create_all()        
ses = Session()  

In [ ]:
ses.add(Car(id=1, name='Audi', price=52642))
ses.commit()
ses.close()

In [ ]:
with Session() as s:
  s.add_all([ 
    Car(id=2, name='Mercedes', price=57127),
    Car(id=3, name='Skoda', price=9000),
    Car(id=4, name='Volvo', price=29000),
    Car(id=5, name='Bentley', price=350000),
    Car(id=6, name='Citroen', price=21000),
    Car(id=7, name='Hummer', price=41400),
    Car(id=8, name='Volkswagen', price=21600)
  ])
  s.commit()

In [ ]:
# Fazendo a query
with Session() as s:
  rs = s.query(Car).all()
  for car in rs:
    print(car.name, car.price)

In [ ]:
with Session() as s:
  rs = s.query(Car).filter(Car.name.like('%en'))
  for car in rs:
    print(car.name, car.price)

In [ ]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

In [ ]:
class Author(Base):
  __tablename__ = "Authors"
 
  author_id = Column(Integer, primary_key=True)
  name = Column(String)  
  books = relationship("Book")

class Book(Base):
  __tablename__ = "Books"
 
  book_id = Column(Integer, primary_key=True)
  title = Column(String)      
  author_id = Column(Integer, ForeignKey("Authors.author_id"))    
  author = relationship("Author")                           

In [ ]:
Base.metadata.create_all()

In [ ]:
with Session() as s:
  res = s.query(Author).filter(Author.name=="Leo Tolstoy").first()
  print(res)
  for book in res.books:
    print (book.title)

In [ ]:
with Session() as s:
  res = s.query(Book).filter(Book.title=="Emma").first()    
  print(res.author.name)